In [ ]:
# default_exp methods

# Hierachical Reconciliation Methods

In [ ]:
#export
from typing import Dict, List

import numpy as np
from statsmodels.stats.moment_helpers import cov2corr

In [ ]:
#hide
from fastcore.test import ExceptionExpected, test_close, test_eq

In [ ]:
#export
def _reconcile(S: np.ndarray, P: np.ndarray, W: np.ndarray, 
               y_hat: np.ndarray, SP: np.ndarray = None):
    if SP is None:
        SP = S @ P
    return np.matmul(SP, y_hat)

In [ ]:
#export
def bottom_up(S: np.ndarray,
              y_hat: np.ndarray,
              idx_bottom: List[int]):
    n_hiers, n_bottom = S.shape
    P = np.zeros_like(S, dtype=np.float32)
    P[idx_bottom] = S[idx_bottom]
    P = P.T
    W = np.eye(n_hiers, dtype=np.float32)
    return _reconcile(S, P, W, y_hat)

In [ ]:
#export
class BottomUp:
    
    def reconcile(self,
                  S: np.ndarray,
                  y_hat: np.ndarray,
                  idx_bottom: np.ndarray):
        return bottom_up(S=S, y_hat=y_hat, idx_bottom=idx_bottom)
    
    __call__ = reconcile

In [ ]:
#hide
S = np.array([
    [1., 1., 1., 1.],
    [1., 1., 0., 0.],
    [0., 0., 1., 1.],
    [0., 1., 0., 0.],
    [1., 0., 0., 0.],
    [0., 0., 1., 0.],
    [0., 0., 0., 1.],
])
h = 10
_y = np.array([10., 5., 4., 2., 1.])
y_bottom = np.vstack([i * _y for i in range(1, 5)])
resids_bottom = y_bottom - np.roll(y_bottom, 1)
resids_bottom[:, 0] = np.nan
y_hat_bottom = np.vstack([i * np.ones(h) for i in range(1, 5)])
idx_bottom = [4, 3, 5, 6]
levels = {'level1': np.array([0]),
          'level2': np.array([2, 1]),
          'level3': idx_bottom}

In [ ]:
#hide
cls_bottom_up = BottomUp()
test_eq(
    cls_bottom_up(S=S, y_hat=S @ y_hat_bottom, idx_bottom=idx_bottom),
    S @ y_hat_bottom
)

In [ ]:
#export
def is_strictly_hierarchical(S: np.ndarray, 
                             levels: Dict[str, np.ndarray]):
    # main idea:
    # if S represents a strictly hierarchical structure
    # the number of paths before the bottom level
    # should be equal to the number of nodes
    # of the previuos level
    levels_ = dict(sorted(levels.items(), key=lambda x: len(x[1])))
    # removing bottom level
    levels_.popitem()
    # making S categorical
    hiers = [np.argmax(S[idx], axis=0) + 1 for _, idx in levels_.items()]
    hiers = np.vstack(hiers)
    paths = np.unique(hiers, axis=1).shape[1] 
    nodes = levels_.popitem()[1].size
    return paths == nodes

In [ ]:
#hide
assert is_strictly_hierarchical(S, levels)
S_non_hier = np.array([
    [1., 1., 1., 1.], # total
    [1., 1., 0., 0.], # city 1
    [0., 0., 1., 1.], # city 2
    [1., 0., 1., 0.], # transgender
    [0., 1., 0., 1.], # no transgender
    [1., 0., 0., 0.], #city 1 - transgender
    [0., 1., 0., 0.], #city 1 - no transgender
    [0., 0., 1., 0.], #city 2 - transgender
    [0., 0., 0., 1.], #city 2 - no transgender
])
levels_non_hier = {
    'Country': np.array([0]),
    'Country/City': np.array([2, 1]),
    'Country/Transgender': np.array([3, 4]),
    'Country-City-Transgender': np.array([5, 6, 7, 8]),
}
assert not is_strictly_hierarchical(S_non_hier, levels_non_hier)

In [ ]:
#export
def top_down(S: np.ndarray, 
             y_hat: np.ndarray,
             y: np.ndarray,
             levels: Dict[str, np.ndarray],
             method: str):
    if not is_strictly_hierarchical(S, levels):
        raise ValueError('Top down reconciliation requires strictly hierarchical structures.')
    
    n_hiers, n_bottom = S.shape
    idx_top = int(S.sum(axis=1).argmax())
    levels_ = dict(sorted(levels.items(), key=lambda x: len(x[1])))
    idx_bottom = levels_[list(levels_)[-1]]
    
    if method == 'forecast_proportions':
        raise NotImplementedError(f'Method {method} not implemented yet')
    else:
        y_top = y[idx_top]
        y_btm = y[idx_bottom]
        if method == 'average_proportions':
            prop = np.mean(y_btm / y_top, axis=1)
        elif method == 'proportion_averages':
            prop = np.mean(y_btm, axis=1) / np.mean(y_top)
        else:
            raise Exception(f'Unknown method {method}')
    P = np.zeros_like(S).T
    P[:, idx_top] = prop
    W = np.eye(n_hiers, dtype=np.float32)
    return _reconcile(S, P, W, y_hat)

In [ ]:
#export
class TopDown:
    
    def __init__(self, method: str):
        self.method = method
    
    def reconcile(self, 
                  S: np.ndarray, 
                  y_hat: np.ndarray,
                  y: np.ndarray,
                  levels: Dict[str, np.ndarray],):
        return top_down(S=S, y_hat=y_hat, y=y, 
                        levels=levels,
                        method=self.method)
    
    __call__ = reconcile

In [ ]:
#hide
for method in ['average_proportions', 'proportion_averages']:
    cls_top_down = TopDown(method=method)
    test_close(
        cls_top_down(
            S=S, 
            y_hat=S @ y_hat_bottom, 
            y=S @ y_bottom, 
            levels=levels
        ),
        S @ y_hat_bottom
    )

In [ ]:
#export
def crossprod(x):
    return x.T @ x

In [ ]:
#export
def min_trace(S: np.ndarray, 
              y_hat: np.ndarray,
              residuals: np.ndarray,
              method: str):
    # shape residuals (obs, n_hiers)
    res_methods = ['wls_var', 'mint_cov', 'mint_shrink']
    if method in res_methods and residuals is None:
        raise ValueError(f"For methods {', '.join(res_methods)} you need to pass residuals")
    n_hiers, n_bottom = S.shape
    if method == 'ols':
        W = np.eye(n_hiers)
    elif method == 'wls_struct':
        W = np.diag(S @ np.ones((n_bottom,)))
    elif method in res_methods:
        n, _ = residuals.shape
        masked_res = np.ma.array(residuals, mask=np.isnan(residuals))
        covm = np.ma.cov(masked_res, rowvar=False, allow_masked=True).data
        if method == 'wls_var':
            W = np.diag(np.diag(covm))
        elif method == 'mint_cov':
            W = covm
        elif method == 'mint_shrink':
            tar = np.diag(np.diag(covm))
            corm = cov2corr(covm)
            xs = np.divide(residuals, np.sqrt(np.diag(covm)))
            xs = xs[~np.isnan(xs).any(axis=1), :]
            v = (1 / (n * (n - 1))) * (crossprod(xs ** 2) - (1 / n) * (crossprod(xs) ** 2))
            np.fill_diagonal(v, 0)
            corapn = cov2corr(tar)
            d = (corm - corapn) ** 2
            lmd = v.sum() / d.sum()
            lmd = max(min(lmd, 1), 0)
            W = lmd * tar + (1 - lmd) * covm
    else:
        raise ValueError(f'Unkown reconciliation method {method}')
    
    eigenvalues, _ = np.linalg.eig(W)
    if any(eigenvalues < 1e-8):
        raise Exception(f'min_trace ({method}) needs covariance matrix to be positive definite.')
        
    R = S.T @ np.linalg.inv(W)
    P = np.linalg.inv(R @ S) @ R
    
    return _reconcile(S, P, W, y_hat)

In [ ]:
#export
class MinTrace:
    
    def __init__(self, method: str):
        self.method = method
        
    def reconcile(self, 
                  S: np.ndarray, 
                  y_hat: np.ndarray,
                  residuals: np.ndarray):
        return min_trace(S=S, y_hat=y_hat, 
                         residuals=residuals,
                         method=self.method)
    
    __call__ = reconcile

In [ ]:
#hide
for method in ['ols', 'wls_struct', 'wls_var', 'mint_shrink']:
    cls_min_trace = MinTrace(method=method)
    test_close(
        cls_min_trace(
            S, 
            S @ y_hat_bottom, 
            np.transpose(S @ resids_bottom)
        ),
        S @ y_hat_bottom
    )
with ExceptionExpected(regex='min_trace (mint_cov)*'):
    cls_min_trace = MinTrace(method='mint_cov')
    cls_min_trace(
        S, 
        S @ y_hat_bottom, 
        np.transpose(S @ resids_bottom)
    )

In [ ]:
#export
def erm(S: np.ndarray,
        y_hat: np.ndarray,
        method: str,
        lambda_reg: float = 1e-2):
    n_hiers, n_bottom = S.shape
    if method == 'exact':
        B = y_hat.T @ S @ np.linalg.inv(S.T @ S).T
        P = B.T @ y_hat.T @ np.linalg.inv(y_hat @ y_hat.T + lambda_reg * np.eye(n_hiers))
    else:
        raise ValueError(f'Unkown reconciliation method {method}')
        
    W = np.eye(n_hiers, dtype=np.float32)
    
    return _reconcile(S, P, W, y_hat)

In [ ]:
#export
class ERM:
    
    def __init__(self, method: str, lambda_reg: float = 1e-2):
        self.method = method
        self.lambda_reg = lambda_reg
        
    def reconcile(self, S: np.ndarray,
                  y_hat: np.ndarray):
        return erm(S=S, y_hat=y_hat, 
                   method=self.method, lambda_reg=self.lambda_reg)
    
    __call__ = reconcile

In [ ]:
#hide
for method in ['exact']:
    cls_erm = ERM(method=method, lambda_reg=1e-3)
    test_close(
        cls_erm(
            S, 
            S @ y_hat_bottom
        ),
        S @ y_hat_bottom
    )